In [1]:
import numpy.matlib 
import numpy as np 
import re

def getxy(nlist, x,y):
    for n in nlist:
        if n.x == x and n.y == y:
            return n
    return False

class node:
    def __init__(self,name,data,x=0, y=0):
        self.node_data = [list(x) for x in data]
        self.node_name = name
        self.x = x
        self.y = y
        self.static = False  # is this locked in place
        self.attached = [False,False,False,False] # who is attached to the top,right,bottom,left
        
    def setStatic(self,v):
        self.static = v
        
    def __str__(self):
        return "\n".join( [ "".join(x) for x in self.node_data ] )
        
    # 0 = top
    # 1 = right
    # 2 = bottom
    # 3 = left
    def get_edge(self,n):
        if n == 0:
            return "".join(self.node_data[0])
        elif n == 1:
            return "".join([ x[-1] for x in self.node_data ])
        elif n == 2:
            return "".join(self.node_data[-1])
        elif n == 3:
            return "".join([ x[0] for x in self.node_data ])

    def flip(self):
        if self.static:
            #print("error {} is static".format( self.node_name))
            return False
        a = np.matrix( self.node_data )
        b = np.fliplr( a)
        self.node_data = b.tolist()
        return True
    
    # https://stackoverflow.com/questions/42519/how-do-you-rotate-a-two-dimensional-array
    def rotate_cw_90(self):
        if self.static:
            #print("error {} is static".format( self.node_name))
            return False
        a = np.matrix( self.node_data )
        b = np.fliplr( a.transpose() )
        self.node_data = b.tolist()
        return True
            

def flipString(s):
    a = np.matrix([list(x) for x in s.strip().split('\n')])
    b = np.fliplr(a).tolist()
    return "\n".join(["".join(x) for x in b])

def rotateStringcw90(s):
    a = np.matrix([list(x) for x in s.strip().split('\n')])
    b = np.fliplr( a.transpose() ).tolist()

    return "\n".join(["".join(x) for x in b])



### Read in files
#input_file="data/day20_sample_1.txt"
input_file="data/day20.txt"

l1 = [ x.strip() for x in open(input_file).readlines()]

## Read in the tiles
tile=""
tiles = {}
connections = {}
for line in l1:
    if line == "":
        continue
    if line.find("Tile") != -1:
        m = re.match("Tile (\d+):", line)
        tile = m[1]
        connections[tile] = []
    else:
        if tile in tiles:
            tiles[tile].append(line)
        else:
            tiles[tile] = [line]

nodes = {}          
for t in tiles:
    nodes[t] = node(t, tiles[t])
    
    
### Process Data
## Build out a row
def build_row(n_in_row):
    
    rows_data = n_in_row.node_data
    rows_names = [n_in_row.node_name]
    
    # build out left
    lnode=n_in_row
    while True:
        found_node  = find_attached_node(lnode, 3)
        if found_node:
            rows_names = [found_node.node_name] + rows_names
            found_node.setStatic(True)
            lnode = found_node
        else:
            # nothing left to find
            break
   
    # build out right 
    rnode=n_in_row
    while True:
        found_node = find_attached_node(rnode, 1) 
        if found_node:
            rows_names = rows_names + [found_node.node_name]
            found_node.setStatic(True)
            rnode = found_node
        else:
            #nothing left to find
            break
            
    return [rows_data, rows_names]

# 0: top
# 1: right
# 2: bottom
# 3: left
def find_attached_node(current_node, side):
    mate_side = [2,3,0,1][side]
    for check_node in nodes:
        check_node = nodes[check_node]
        if check_node == current_node:
            continue
        for flip in [0,1]: # try it flipped and unfliped
            for rot in [0,1,2,3]: # try every rotation
                if current_node.get_edge(side) == check_node.get_edge(mate_side):
                    check_node.setStatic(True)
                    return check_node
                check_node.rotate_cw_90()
            check_node.flip()
    # nothing found 
    return False

first_node = next(enumerate(nodes))[1]
nodes[first_node].setStatic(True)
rows = [ build_row(nodes[first_node])[1] ]


top = find_attached_node( nodes[first_node], 0)
bottom = find_attached_node( nodes[first_node], 2)

while True:
    
    ## if top and bottom are empty, then we are done
    if not top and not bottom:
        break
    
    if top:
        rows = [build_row(top)[1]] + rows
        top = find_attached_node( nodes[ rows[0][0] ] ,0 )
    if bottom:
        rows = rows + [build_row(bottom)[1]] 
        bottom = find_attached_node(nodes[ rows[-1][0] ],2)

aa = ""
for row in rows:
    for i in range(1,9):
        aa +=  "".join([ "".join(x[i][1:9]) for x in [ nodes[z].node_data for z in row]] )
        aa += "\n"
print(rows)
print(aa)


[['1321', '3373', '2213', '1061', '3851', '1117', '2081', '2087', '1811', '3011', '3769', '1613'], ['1069', '3709', '3343', '3119', '2293', '1481', '1979', '3319', '1483', '1063', '2381', '1361'], ['1009', '2137', '3257', '2843', '1213', '1289', '1277', '1399', '3109', '3559', '1823', '3371'], ['1877', '2539', '2777', '2659', '2719', '3449', '1217', '1999', '3049', '2017', '2521', '2593'], ['1291', '2341', '1657', '2441', '2003', '1579', '1571', '1459', '1249', '1181', '1279', '2153'], ['2143', '3061', '3331', '2789', '3391', '2243', '3491', '2857', '3037', '2687', '2551', '3019'], ['2411', '2711', '3847', '2269', '3571', '3533', '1879', '2503', '1229', '1499', '2803', '3659'], ['1663', '1559', '1637', '1759', '1097', '2423', '2207', '2731', '3701', '3347', '3677', '2467'], ['1523', '3947', '1451', '2251', '1433', '2677', '3739', '3803', '3923', '2027', '3457', '3863'], ['1013', '3889', '3359', '1987', '3989', '2029', '2383', '1531', '2909', '2851', '1609', '1753'], ['3187', '2011', '1

In [18]:
import regex as re

def findMonsters(map_data):
    ret = []
    regex1='#....##....##....###'
    regex2='.#..#..#..#..#..#...'

    md = map_data.strip().split('\n')
    for y in range(0,len(md)):
        for m in re.finditer(regex1, md[y], overlapped=True):  # YOU NEED TO FIND OVERLAPPING MATCHES
            x = m.start()
            if re.match(regex2, md[y+1][x:]):
                if md[y-1][x + (len(regex1)-2) ] == "#":
                    ret.append([x,y])

    return ret

import copy

p1 = copy.deepcopy(aa)
seamonstersize=15
for f in [0,1]:
    p1 = flipString(p1)
    for r in [0,1,2,3]:
        p1 = rotateStringcw90(p1)
        seaMonsters = len(findMonsters(p1))
        if seaMonsters > 0:
            seaTiles = seaMonsters * seamonstersize
            ans = p1.count('#') - seaTiles
            print("answer:",ans, seaMonsters)
    

answer: 2323 25
